## JOIN
Neste notebook veremos como unir dados de mais de um Dataframe

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, lit, expr

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
# Inicia uma sparkSession
spark = SparkSession \
    .builder \
    .appName("Meu curso de pyspark") \
    .getOrCreate()

### Criando alguns Dataframes de exemplo

In [7]:
# Define as linhas
newRows = [(1, "Mateus", 0, [300]),(2, "Patricia", 2, [500, 400]),(3, "Rafaela", 1, [500]),(4, "Henrique", -1, [-1])]

# Cria um RDD de Rows
parallelizedRows = spark.sparkContext.parallelize(newRows)

# Cria um dataframe a partir do RDD que criamos anteriormente
pessoa = spark.createDataFrame(parallelizedRows, ["id", "nome", "curso", "id_funcoes"])

# Mostra as informações do dataframe
pessoa.show()

+---+--------+-----+----------+
 id| nome|curso|id_funcoes|
+---+--------+-----+----------+
 1| Mateus| 0| [300]|
 2|Patricia| 2|[500, 400]|
 3| Rafaela| 1| [500]|
 4|Henrique| -1| [-1]|
+---+--------+-----+----------+

In [8]:
# Define as linhas
newRows = [(0, "Bacharelado"),(1, "Mestrado"),(2, "Doutorado"),(3, "Tecnólogo")]

# Cria um RDD de Rows
parallelizedRows = spark.sparkContext.parallelize(newRows)

# Cria um dataframe a partir do RDD que criamos anteriormente
curso = spark.createDataFrame(parallelizedRows, ["id", "curso"])

# Mostra as informações do dataframe
curso.show()

+---+-----------+
 id| curso|
+---+-----------+
 0|Bacharelado|
 1| Mestrado|
 2| Doutorado|
 3| Tecnólogo|
+---+-----------+

In [9]:
# Define as linhas
newRows = [(500, "Gerente"),(400, "Dono"),(300, "Funcionário")]

# Cria um RDD de Rows
parallelizedRows = spark.sparkContext.parallelize(newRows)

# Cria um dataframe a partir do RDD que criamos anteriormente
funcao = spark.createDataFrame(parallelizedRows, ["id", "funcao"])

# Mostra as informações do dataframe
funcao.show()

+---+-----------+
 id| funcao|
+---+-----------+
500| Gerente|
400| Dono|
300|Funcionário|
+---+-----------+

### Inner join

In [11]:
joinExpression = pessoa.curso == curso.id
joinType = "inner"
pessoa.join(curso, joinExpression, joinType).show()

+---+--------+-----+----------+---+-----------+
 id| nome|curso|id_funcoes| id| curso|
+---+--------+-----+----------+---+-----------+
 1| Mateus| 0| [300]| 0|Bacharelado|
 3| Rafaela| 1| [500]| 1| Mestrado|
 2|Patricia| 2|[500, 400]| 2| Doutorado|
+---+--------+-----+----------+---+-----------+

### Left join

In [13]:
joinExpression = pessoa.curso == curso.id
joinType = "left"
pessoa.join(curso, joinExpression, joinType).show()

+---+--------+-----+----------+----+-----------+
 id| nome|curso|id_funcoes| id| curso|
+---+--------+-----+----------+----+-----------+
 1| Mateus| 0| [300]| 0|Bacharelado|
 3| Rafaela| 1| [500]| 1| Mestrado|
 2|Patricia| 2|[500, 400]| 2| Doutorado|
 4|Henrique| -1| [-1]|null| null|
+---+--------+-----+----------+----+-----------+

### Right join

In [15]:
joinExpression = pessoa.curso == curso.id
joinType = "right"
pessoa.join(curso, joinExpression, joinType).show()

+----+--------+-----+----------+---+-----------+
 id| nome|curso|id_funcoes| id| curso|
+----+--------+-----+----------+---+-----------+
 1| Mateus| 0| [300]| 0|Bacharelado|
 3| Rafaela| 1| [500]| 1| Mestrado|
null| null| null| null| 3| Tecnólogo|
 2|Patricia| 2|[500, 400]| 2| Doutorado|
+----+--------+-----+----------+---+-----------+

### Outer join

In [17]:
joinExpression = pessoa.curso == curso.id
joinType = "outer"
pessoa.join(curso, joinExpression, joinType).show()

+----+--------+-----+----------+----+-----------+
 id| nome|curso|id_funcoes| id| curso|
+----+--------+-----+----------+----+-----------+
 1| Mateus| 0| [300]| 0|Bacharelado|
 3| Rafaela| 1| [500]| 1| Mestrado|
null| null| null| null| 3| Tecnólogo|
 2|Patricia| 2|[500, 400]| 2| Doutorado|
 4|Henrique| -1| [-1]|null| null|
+----+--------+-----+----------+----+-----------+

### Cross join

In [19]:
pessoa.crossJoin(curso).show()

+---+--------+-----+----------+---+-----------+
 id| nome|curso|id_funcoes| id| curso|
+---+--------+-----+----------+---+-----------+
 1| Mateus| 0| [300]| 0|Bacharelado|
 1| Mateus| 0| [300]| 1| Mestrado|
 1| Mateus| 0| [300]| 2| Doutorado|
 1| Mateus| 0| [300]| 3| Tecnólogo|
 2|Patricia| 2|[500, 400]| 0|Bacharelado|
 2|Patricia| 2|[500, 400]| 1| Mestrado|
 2|Patricia| 2|[500, 400]| 2| Doutorado|
 2|Patricia| 2|[500, 400]| 3| Tecnólogo|
 3| Rafaela| 1| [500]| 0|Bacharelado|
 3| Rafaela| 1| [500]| 1| Mestrado|
 3| Rafaela| 1| [500]| 2| Doutorado|
 3| Rafaela| 1| [500]| 3| Tecnólogo|
 4|Henrique| -1| [-1]| 0|Bacharelado|
 4|Henrique| -1| [-1]| 1| Mestrado|
 4|Henrique| -1| [-1]| 2| Doutorado|
 4|Henrique| -1| [-1]| 3| Tecnólogo|
+---+--------+-----+----------+---+-----------+

### Left semi join
Lista todos os registros do dataframe da esquerda que possuem match no dataframe da direita.<br>
Podemos olhar para este JOIN como uma espécie de filtro (não é efetivamente um JOIN como estamos acostumados)

In [21]:
joinExpression = pessoa.curso == curso.id
joinType = "left_semi"
pessoa.join(curso, joinExpression, joinType).show()

+---+--------+-----+----------+
 id| nome|curso|id_funcoes|
+---+--------+-----+----------+
 1| Mateus| 0| [300]|
 3| Rafaela| 1| [500]|
 2|Patricia| 2|[500, 400]|
+---+--------+-----+----------+

### Left anti join
É o oposto do SEMI JOIN, no sentido de que apenas os registros que NÃO possuem match no dataframe da direta serão retornados

In [23]:
joinExpression = pessoa.curso == curso.id
joinType = "left_anti"
pessoa.join(curso, joinExpression, joinType).show()

+---+--------+-----+----------+
 id| nome|curso|id_funcoes|
+---+--------+-----+----------+
 4|Henrique| -1| [-1]|
+---+--------+-----+----------+

### JOIN em tipos de dados complexos

In [25]:
pessoa.withColumnRenamed("id", "pessoaId").join(funcao, expr("array_contains(id_funcoes, id)")).show()

+--------+--------+-----+----------+---+-----------+
pessoaId| nome|curso|id_funcoes| id| funcao|
+--------+--------+-----+----------+---+-----------+
 1| Mateus| 0| [300]|300|Funcionário|
 2|Patricia| 2|[500, 400]|500| Gerente|
 2|Patricia| 2|[500, 400]|400| Dono|
 3| Rafaela| 1| [500]|500| Gerente|
+--------+--------+-----+----------+---+-----------+

### Tratado possível problema de nome de coluna duplicado

In [27]:
joinExpression = pessoa.curso == curso.id
joinType = "inner"
resultado = pessoa.join(curso, joinExpression, joinType)

**Note que o resultado é um dataframe com duas colunas com o mesmo nome (curso)**

In [29]:
resultado.show(5)

+---+--------+-----+----------+---+-----------+
 id| nome|curso|id_funcoes| id| curso|
+---+--------+-----+----------+---+-----------+
 1| Mateus| 0| [300]| 0|Bacharelado|
 3| Rafaela| 1| [500]| 1| Mestrado|
 2|Patricia| 2|[500, 400]| 2| Doutorado|
+---+--------+-----+----------+---+-----------+

**Erro: Neste cenário, o comando abaixo não vai funcionar porque spark não saberá quais das duas colunas devem ser retornadas**

In [31]:
resultado.select("curso").show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2586057201697803> in <module> 
 ----> 1 resultado . select ( "curso" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1437 [ Row ( name = u'Alice' , age = 12 ) , Row ( name = u'Bob' , age = 15 ) ] 
 1438 """
 -> 1439 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1440 return DataFrame ( jdf , self . sql_ctx ) 
 1441 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 132 # Hide where the exception came from that shows a non-Pythonic 
 133 # JVM exception message. 
 --> 134 raise_from ( converted ) 
 135 else : 
 136 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Reference 'curso' is ambiguous, could be: curso, curso.;

**Podemos excluir uma das colunas após a junçao**

In [33]:
resultado = pessoa.join(curso, joinExpression, joinType).drop(pessoa["curso"])
resultado.select("curso").show()

+-----------+
 curso|
+-----------+
Bacharelado|
 Mestrado|
 Doutorado|
+-----------+

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/